# CH-08 建構投資組合：讓 AI 輔助選股

## 8-1 建構投資組合

### 1️⃣ 匯入套件 & 掛載雲端硬碟



In [ ]:
!pip install gdown
import gdown
import os
import datetime as dt
import time
from IPython.display import display
import pandas as pd
from google.colab import drive
from tqdm import tqdm
drive.mount('/content/drive')

Mounted at /content/drive


### 2️⃣ 下載資料庫
若資料庫未下載成功或出現 bug，請手動點擊連結下載，並放入 **我的雲端硬碟>StockGPT** 資料夾中

下載連結：https://drive.usercontent.google.com/download?id=1S5JE9ZF2hohRpvO8FikgLhQmN2DJrVgW

In [ ]:
# 指定下載路徑
!mkdir -p "/content/drive/MyDrive/StockGPT/"
output_path = '/content/drive/MyDrive/StockGPT/'

# 檢查資料庫是否存在
stock_db_path = output_path + 'stock.db'
if not os.path.exists(stock_db_path):
    print("下載資料庫中...")
    id = '1S5JE9ZF2hohRpvO8FikgLhQmN2DJrVgW'

    gdown.download(id=id, output=stock_db_path)
    print("下載完成")
else:
    print("無需下載")

無需下載


### 3️⃣ 安裝旗標套件

In [ ]:
!git clone https://github.com/FlagTech/F3933.git
%cd F3933
!pip install -r requirements.txt
from Ch06 import StockAnalysis, StockInfo
from Ch07 import PdfLoader
from Stock_DB import StockDB
%cd ..

fatal: destination path 'F3933' already exists and is not an empty directory.
/content/F3933
/content


### 4️⃣ 更新資料庫
資料庫更新需耗費較長時間, 若無法更新，請刪除我的雲端硬碟>StockGPT>stock.db 檔案，並至第二個儲存格的連結手動下載

In [ ]:
stock_db = StockDB()
stock_db.renew()
stock_db.close()

###5️⃣ 檢視資料表格式

In [ ]:
# 從資料庫中取得表格
stock_db = StockDB()
df_company = stock_db.get("公司")
df_daily = stock_db.get("日頻",psdate=True)
df_quarterly = stock_db.get("季頻",psdate=True)

# 去除空值
df_company = df_company.dropna()
df_daily = df_daily.dropna()
df_quarterly = df_quarterly.dropna()

# 檢視資料
display(df_company.head())
display(df_daily.head())
display(df_quarterly.head())

,股號,股名,產業別,股本,市值
0,1101,台泥,水泥工業,7.136180e+09,2.501231e+11
1,1102,亞泥,水泥工業,3.546560e+09,1.425717e+11
2,1103,嘉泥,水泥工業,6.588940e+08,1.242015e+10
3,1104,環泥,水泥工業,6.732170e+08,1.908570e+10
4,1108,幸福,水泥工業,4.047380e+08,6.212728e+09


,股號,日期,開盤價,最高價,最低價,收盤價,還原價,成交量,殖利率,日本益比,股價淨值比,三大法人買賣超股數,融資買入,融卷賣出
0,1101,2015-01-05,31.923370,31.923370,31.482035,31.629147,20.498331,3847400.0,5.35,14.88,1.42,-1005483,211.0,20.0
1,1101,2015-01-06,31.261366,31.334923,30.967140,31.077475,20.140799,10386622.0,5.44,14.62,1.40,-2524874,208.0,32.0
2,1101,2015-01-07,31.077475,31.298143,30.783251,30.967140,20.069292,11841293.0,5.46,14.57,1.39,-5742000,264.0,13.0
3,1101,2015-01-08,31.077475,31.371700,30.967140,31.077475,20.140799,11815462.0,5.44,14.62,1.40,-5120668,245.0,38.0
4,1101,2015-01-09,31.114252,31.334923,30.746471,30.746471,19.926281,14240820.0,5.50,14.46,1.38,-7993555,566.0,56.0


,股號,日期,營業收入,營業費用,稅後淨利,每股盈餘
0,1101,2024-03-01,25544599.0,2687318.0,2239346.0,0.26
1,1101,2023-12-01,28347871.0,2871134.0,2499914.0,0.25
2,1101,2023-09-01,27002293.0,2712611.0,2292372.0,0.17
3,1101,2023-06-01,27668242.0,2616710.0,4206683.0,0.45
4,1101,2023-03-01,26295929.0,2303154.0,1005644.0,0.20


###6️⃣ 輸入 OpenAI API KEY & 建立物件

In [ ]:
import getpass
openai_api_key = getpass.getpass("請輸入金鑰：")
six = StockAnalysis(openai_api_key)
seven = PdfLoader(openai_api_key)

請輸入金鑰：··········


### 7️⃣ AI 自動化選股機器人

<輸入範例>
```
請選出近一週漲幅最高的10檔股票
請選出大市值股(前10%)且近期營收成長最高的10檔股票
請選出半導體業且近期每股盈餘成長率最高的10檔股票
請選出近一年股價淨值比最低的10檔股票
請從近一個月成交量前10%的股票中找出日本益比最低的5檔不同的股票
```


In [ ]:
# 輸入使用者需求
user_msg = input("請輸入選股需求:")

# 自動處理 3 個資料表並生成程式碼
history, code_str = six.ai_helper(user_msg)

# AI 自動選股 & 除錯
success = False
for _ in range(3):
    try:
        # 新建 df 表格
        df1 = df_company
        df2 = df_daily
        df3 = df_quarterly

        print(code_str)
        exec(code_str)
        new_df = calculate(df1, df2, df3)
        success = True
        break
    except Exception as e:
        print(f"AI 除錯中...")
        code_str = six.ai_debug(history, code_str, str(e))
        print(code_str)
        print("-------------------------")

if not success:
    print("請更換或重新輸入選股需求")
else:
    display(new_df)

請輸入選股需求:請從近一個月成交量前10%的股票中找出日本益比最低的5檔不同的股票

import pandas as pd

def calculate(table_company, table_daily, table_quarterly):
    table_daily['日本益比'] = pd.to_numeric(table_daily['日本益比'], errors='coerce')
    table_daily['成交量'] = pd.to_numeric(table_daily['成交量'], errors='coerce')
    recent_month_volume = table_daily['日期'].max() - pd.DateOffset(months=1)
    top_10_percent_volume = table_daily[table_daily['日期'] > recent_month_volume].nlargest(int(len(table_daily) * 0.1), '成交量')
    top_10_percent_volume = top_10_percent_volume.groupby('股號').last().reset_index()
    top_5_low_pe_stocks = top_10_percent_volume.nsmallest(5, '日本益比')

    return top_5_low_pe_stocks[['股號', '股名', '日本益比']].drop_duplicates()

AI 除錯中...
import pandas as pd

def calculate(table_company, table_daily, table_quarterly):
    table_daily['日本益比'] = pd.to_numeric(table_daily['日本益比'], errors='coerce')
    table_daily['成交量'] = pd.to_numeric(table_daily['成交量'], errors='coerce')
    recent_month_volume = table_daily['日期'].max(

,股號,日本益比
421,2603,1.59
13,1218,1.93
426,2609,2.17
351,2443,2.33
215,2025,2.65


## 8-2 AI 趨勢報告推薦系統

### 8️⃣ 取得個股分析報告

In [ ]:
reply = six.stock_gpt(stock_id="2330")
print(reply)

[*********************100%%**********************]  1 of 1 completed
台積電（2330-TW）近期的趨勢報告如下：

1. 股價漲幅分析：
根據提供的近期價格資訊，台積電的股價在2023年10月30日至11月13日之間有所波動。從收盤價的變化來看，股價從532.0元上升至573.0元，漲幅約為7.72%。每日報酬方面，股價的波動幅度較小，整體呈現穩定的趨勢。

2. 基本面分析：
根據提供的每季營收資訊，台積電在2023年第三季度的營收成長率為1136.03%，顯示公司業績有顯著的增長。然而，需要注意的是，前三個季度的營收成長率分別為-0.05%、-0.19%和0.02%，顯示公司在前兩個季度的業績表現較為疲弱。此外，EPS（每股盈餘）在最近四個季度中呈現波動，但整體趨勢仍然穩定。

3. 新聞資訊分析：
根據提供的近期新聞資訊，台積電的股價受到了一些影響。例如，有報導指出台積電的五大客戶追單，其中包括蘋果、超威、博通等重量級客戶，這可能對台積電的業績產生積極影響。此外，還有一些其他股票的漲跌速報，但這些報導對台積電的整體趨勢影響有限。

綜合以上分析，台積電的股價在近期呈現穩定的上升趨勢，並且公司的基本面表現也相對穩定。然而，需要注意的是，公司在前兩個季度的營收成長率較低，可能需要密切關注業績的發展。此外，公司與重要客戶的合作關係對業績也具有重要影響，需要關注相關新聞資訊的變化。

以上報告僅供參考，投資者在做出任何投資決策前應進一步進行自己的研究和分析。


### 9️⃣ 收集多檔股票的趨勢報告
10 檔股票約會跑 5 分鐘, 若跑太久為 API 當掉

In [ ]:
# 建立股票清單
stock_list = ['2364', '2535', '3041', '5215', '2363',
              '1568', '2369', '2816', '9955', '2233']

# 設定儲存路徑
today_time = dt.date.today().strftime("%Y%m%d")
path = '/content/drive/MyDrive/StockGPT/TrendReport/'
os.makedirs(path, exist_ok=True)

# 建立多檔股票的趨勢報告並儲存
content = {}

for stock in stock_list:
  file_path = f"{path}trend_{stock}_{today_time}.txt"

  if os.path.exists(file_path):
    print(f"{stock} 檔案已存在")
  else:
    with open(file_path, "w", encoding="utf-8") as f:
      f.write(six.stock_gpt(stock_id=stock))

  with open(file_path, "r", encoding="utf-8") as f:
      content[stock] = f.read()


2364 檔案已存在
2535 檔案已存在
3041 檔案已存在
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


### 🔟 推薦出一檔股票

In [ ]:
def stock_choice(data):
    # 設定提示模板
    msg = [{
        "role": "system",
        "content":
        "你現在是一位專業的證券分析師, 你會針對各股的專業趨勢分析報告, \n\
         選擇出最適合投資的一檔股票,並說明選擇它的理由。"
    }, {
        "role": "user",
        "content": str(data)
    }]

    reply_data = six.get_reply(msg)
    return reply_data

print(stock_choice(content))

根據提供的資料，我對以上股票的趨勢進行了分析。在這些股票中，我認為科嘉-KY（代號5215-TW）是一檔值得投資的股票。

首先，科嘉-KY的股價在近期呈現波動，但整體呈現上升趨勢。雖然股價波動較大，但公司在市場上仍被認為是值得持續關注的強勢股之一。

其次，從基本面來看，科嘉-KY在2023年6月30日的營收成長率為46%，顯示公司營收呈現正向增長。儘管在此之前的兩個季度營收成長率呈現下降趨勢，但整體來說，公司的營收表現穩定。

最後，根據新聞資訊，科嘉-KY在市場上被認為是「疾風下的勁草」，並且被列為值得持續關注的強勢股之一。這表明市場對於公司的潛在價值持有樂觀態度。

綜合以上分析，我認為科嘉-KY是一檔值得投資的股票。公司的營收表現穩定，並且在市場上受到肯定。投資者應該密切關注公司的營收和盈利表現，並謹慎評估投資風險。希望這份分析對您有所幫助。


### 1️⃣1️⃣ 推薦股票的評分排序

In [ ]:
def stock_sort(data):
    # 設定提示模板
    msg = [{
        "role": "system",
        "content":
        "你現在是一位專業的股票分析師, 會根據各股的專業趨勢分析報告去評斷\
         適不適合投資, 並給予0-100之間的評分。\n\
         以 50 分為基準, 有任何正面消息可以加分如:\n\
         股價整體上升、法人買超、營收成長上升、新聞有正面消息；\n\
         若有任何負面消息必須扣分如:\n\
         股價整體下降、法人賣超、營收成長下降、新聞有負面消息。\n\
         最後請將所有股票依照評分排序出來。"
    }, {
        "role": "user",
        "content": str(data)
    }]
    reply_data = six.get_reply(msg)
    return reply_data

print(stock_sort(content))

根據提供的資料，我將對各股票的趨勢進行分析並給予評分。

1. 2364 (倫飛): 股價波動較大，營收成長率為6%，但EPS季增率波動較大，且受市場環境和產業消息影響。評分: 45

2. 2535 (未提供資料): 無法提供評分。

3. 3041 (未提供資料): 無法提供評分。

4. 5215 (科嘉-KY): 股價波動較大，營收成長率在不同季度呈現波動，且市場上被認為是值得持續關注的強勢股之一。評分: 55

5. 2363 (矽統): 股價呈現明顯的下跌趨勢，營收成長率有一定程度的增長，但EPS季增率波動較大，且受外資賣超情況影響。評分: 40

6. 1568 (倉佑): 股價呈現下跌趨勢，營收成長率輕微增長，但EPS季增率呈現波動，且受多方面因素影響。評分: 42

7. 2369 (菱生): 股價呈現下跌趨勢，營收成長率有所增長，但EPS呈現下滑趨勢，且受市場情緒影響。評分: 45

8. 2816 (旺旺保): 股價波動較大，營收成長率輕微增長，但EPS季增率呈現下降，且受多項因素影響。評分: 48

9. 9955 (佳龍公司): 股價呈現下跌趨勢，營收成長不穩定，且EPS波動較大，受多項因素影響。評分: 40

10. 2233 (宇隆): 股價波動較大，營收有一定程度的增長，且受營運週轉資金需求和產業投資價值影響。評分: 50

根據以上分析，我將這些股票依照評分排序如下：
1. 2816 (旺旺保) - 48
2. 2364 (倫飛) - 45
3. 2369 (菱生) - 45
4. 2233 (宇隆) - 50
5. 5215 (科嘉-KY) - 55
6. 1568 (倉佑) - 42
7. 2363 (矽統) - 40
8. 9955 (佳龍公司) - 40


## 8-3 AI 年報分析推薦系統

### 1️⃣2️⃣ 取得多檔股票的年報檔案

In [ ]:
# 建立股票清單
stock_list = ['2364', '2535', '3041', '5215', '2363',
              '1568', '2369', '2816', '9955', '2233']

# 取得並儲存年報資料
for stock in stock_list:
  if not os.path.exists(f'/content/drive/MyDrive/StockGPT/PDF/113_{stock}.pdf'):
    try:
        seven.annual_report(stock,'113')
    except:
        time.sleep(10)
        while True:
            try:
                seven.annual_report(stock,'113')
                break
            except:
                time.sleep(10*2)

### 1️⃣3️⃣ 建立向量資料庫
10 檔股票約需 20 分鐘

In [ ]:
db_list=[]
for i in stock_list:
    if not os.path.exists(
        f'/content/drive/MyDrive/StockGPT/DB/113_{i}'):
        print('start',i)
        db_list.append(
            seven.pdf_loader(
                f'/content/drive/MyDrive/StockGPT/PDF/113_{i}.pdf',
                600, 60))

### 1️⃣4️⃣ 讀取向量資料庫

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db_list=[]
for i in stock_list:
    db_list.append(
        FAISS.load_local(
          folder_path=f'/content/drive/MyDrive/StockGPT/DB/113_{i}',
          embeddings=OpenAIEmbeddings()
          ))

### 1️⃣5️⃣ 建立問題串列關鍵字

In [ ]:
key_word = ['公司的財務健全度為何？',
            '營運策略和市場定位是什麼？',
            '面對哪些主要風險和挑戰？',
            '公司在研發和創新方面有哪些成就和計劃？',
            '公司治理結構和管理層的組成是？']

### 1️⃣6️⃣ 取得相關資料 & 建立分析機器人

In [ ]:
# 建立從向量資料庫取得相關資訊函式
def generate_data(db, key_word):
    results = []
    for word in key_word:
        results.append(seven.analyze_chain(db, word))
    return results

# 建立年報分析機器人
def stock_report_summary(data):

    msg = [{
        "role": "system",
        "content": "你現在是一位專業的年報分析師,\n\
                    你會針對年報報告彙整出一份專業的分析報告。\n\
                    請以詳細、嚴謹及專業的角度撰寫此報告,並提及重要的數字\
                    ,reply in 繁體中文"
    }, {
        "role": "user",
        "content": str(data)
    }]

    reply_data = six.get_reply(msg)

    return reply_data

### 1️⃣7️⃣ 多檔股票的年報分析報告
10 檔股票約需執行 40 分鐘

In [ ]:
# 年報分析檔案的儲存路徑
path = "/content/drive/MyDrive/StockGPT/AnnualReport/"
os.makedirs(path, exist_ok=True)


# 建立多檔股票的年報分析報告並儲存
content = {}

for stock in tqdm(stock_list, total=len(stock_list)):
  file_path = f"{path}annual_{stock}_113.txt"

  if os.path.exists(file_path):
    print(f"{stock} 檔案已存在")
  else:

      db = db_list[stock_list.index(stock)]
      report = stock_report_summary(generate_data(db, key_word))

      with open(file_path, "w", encoding="utf-8") as f:
        f.write(report)

  # 取得股名
  name = six.stock_info.get_stock_name(stock, six.name_df)

  with open(file_path, "r", encoding="utf-8") as f:
      content[f"{name}({stock})"] = f.read()

  0%|          | 0/10 [00:00<?, ?it/s]

2364 檔案已存在


 10%|█         | 1/10 [00:00<00:04,  2.07it/s]

2535 檔案已存在


 20%|██        | 2/10 [00:00<00:03,  2.56it/s]

...

 80%|████████  | 8/10 [00:02<00:00,  2.68it/s]

9955 檔案已存在


 90%|█████████ | 9/10 [00:03<00:00,  2.46it/s]

2233 檔案已存在


100%|██████████| 10/10 [00:03<00:00,  2.60it/s]


### 1️⃣8️⃣ 根據分析結果推薦出一檔股票

In [ ]:
print(stock_choice(content))

根據所提供的年報分析報告，我會選擇倫飛(2364)作為最適合投資的一檔股票。以下是我選擇它的理由：

1. 財務比率表現良好：根據年報的分析，倫飛公司在財務比率方面表現良好，包括利息保障倍數、資產報酬率、權益報酬率、純益率等指標均呈現上升趨勢。這顯示公司有良好的獲利能力和資產運用效率。

2. 公司努力和措施：倫飛公司積極強化董事會職能、加強公司治理，並重視產品創新和研發，以提高競爭力和市場佔有率。這些努力有助於提高公司的透明度、獲利能力和競爭力。

3. 現金流量允許比率上升：根據年報的分析，倫飛公司的現金流量允許比率上升，顯示公司有足夠的現金流量來應對短期債務，這有助於確保公司的財務穩健性。

綜合以上分析，倫飛公司在財務表現、公司治理和市場競爭力方面均表現良好，並且有積極的發展策略和措施。因此，我認為倫飛(2364)是一檔值得投資的股票。


### 1️⃣9️⃣ 年報分析報告評分排序

In [ ]:
def stock_report_sort(data):
    # 設定提示模板
    msg = [{
        "role": "system",
        "content":
        "你現在是一位專業的股票分析師, 會根據各股的年報分析報告去評斷\
         適不適合投資, 並給予0-100之間的評分。\n\
         以 50 分為基準, 有任何正面消息可以加分如:\n\
         財務呈現穩定增長的趨勢, 具有競爭優勢；\n\
         若有任何負面消息必須扣分如:\n\
         財務呈現下降的趨勢, 不具有競爭優勢。\n\
         最後請將10檔股票依照評分排序出來。"
    }, {
        "role": "user",
        "content": str(data)
    }]
    reply_data = six.get_reply(msg)
    return reply_data

print(stock_report_sort(content))